# Thermal battery - paper 1

This notebook conatins the codes that generate the final figures for publication!

# Surfplot of RRPs for SA and NSW

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

import datetime
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm, dates
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import package.get_NEM_data as gd
from matplotlib.ticker import FormatStrFormatter
from projdirs import paperdir
from pandas.plotting import register_matplotlib_converters

PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

sep = ['\\','/'][sys.platform=='linux']

# choose the state and date here
font_size = 14

def f(x,y):
    Index = (datetime.datetime.combine(x, y))
    if Index in data.index:
        return np.array(data.loc[Index]['spot_price'])
    else:
        return np.array(data.loc[Index-pd.Timedelta('1h')]['spot_price'])

def ff(x):
    return x.hour*60+x.minute

# plt.close("all")
fig = plt.figure(figsize=(16,12))
fig.tight_layout()
fig.subplots_adjust(left=0.0, bottom=0, right=1, top=1, wspace=0.1, hspace=0.1)

i = 0
for Region in ['SA', 'BGE']:
      for Year in [2010,2019]:
            i+=1
            if Region in NEM:
                price_spot=gd.load_rrp_cal_stamped(Year, Region)
                price_spot['year']=Year
                price_spot.spot_price = price_spot[['spot_price','year']].apply(
                    lambda x: gd.convert_curr(*x ),axis=1)
                price_spot = price_spot.drop('year', axis=1)
                
            elif Region in PJM:
                price_spot=gd.load_lmp_zone_year(Region, Year, lmp_type='rt').reset_index()
                price_spot.columns = [ 'date_time', 'spot_price']
                price_spot['region'] = Region
                
            data = price_spot.set_index('date_time')
            x1 = pd.date_range(data.index.date.min(),
                               data.index.date.max(), freq='1D').date #2D
            y1 = pd.date_range('00:00:00','23:00:00', freq='60min').time
      
            x2, y2 = np.meshgrid(x1,y1)
      
            g = np.vectorize(f)
            gg = np.vectorize(ff)
            z = g(x2,y2)
            
            z[z<-50]=-50
            z[z>300]=300

            ax = fig.add_subplot(220+i, projection='3d', proj_type = 'ortho')
            
            
            surf = ax.plot_surface(dates.date2num(x2), gg(y2), z, rstride=1, cstride = 1,
                           edgecolor='k', linewidth=0.2, cmap=cm.jet)
                        
            x_tick_range = pd.date_range(data.index.date.min(),
                       data.index.date.max(), freq='1D').date
            ax.set_xlim(x_tick_range.min(), x_tick_range.max())
            ax.xaxis.set_tick_params(rotation=90 )
            
            ax.set_zticklabels(ax.axes.get_zticks(), horizontalalignment= 'left')
            ax.zaxis.set_major_formatter(FormatStrFormatter('%.0f'))
            
            ax.axes.set_yticks(np.arange(0,25*60,240))
            ax.axes.set_ylim([0,1450])
            ytick_range = ax.axes.get_yticks()
            m = len(ax.axes.get_yticks())
            ylabels = pd.date_range(start = '2000-01-01', end = '2000-01-02',  periods=m)
            ylabels = [datetime.datetime.strftime(Label, '%H:%M') for Label in ylabels]
            xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
            ax.set_yticklabels(ylabels,rotation=-90, va='top', ha='right')
            ax.set_xticklabels(xlabels, va='top', ha='right')
            
            ax.set_zlabel('Spot price [$/MWh]', fontsize=font_size, labelpad=15)
            ax.axes.invert_xaxis()
            ax.xaxis.set_tick_params(labelsize=font_size,rotation=45, pad=0.1)
            ax.yaxis.set_tick_params(labelsize=font_size)
            ax.zaxis.set_tick_params(labelsize=font_size)
            ax.set_zlim([-50,300])
#             cbar = fig.colorbar(surf, ax=ax, shrink=0.5, pad=0)
#             cbar.ax.tick_params(labelsize=font_size)
            ax.set_title('%s - %d' %(Region,Year), {
                        'fontsize': font_size,
#                         'fontweight' : '',
                        'verticalalignment': 'top'
                        })
                    
            ax.view_init(elev=30)
            ax.dist = 11

plt.show()
# savedir = paperdir + 'Pictures%sRRP annual%sSurfplots%s'%(sep,sep,sep)
# plt.savefig(savedir + "RRP_surf.png", format='png', dpi=300,  bbox_inches = "tight")
# plt.savefig(savedir + "RRP_surf.svg", format='svg', bbox_inches = "tight")


In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

import datetime
import pandas as pd
import package.get_NEM_data as gd
import plotly.graph_objects as go

price_spot=gd.load_rrp_cal_stamped(2018, 'VIC')

Fig = go.Figure()

Fig.add_trace(go.Scatter(x = price_spot.date_time,
                         y = price_spot.spot_price,
                         line_shape='hv')
             )
Fig.show()


# RRP histograms

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import paperdir
import numpy as np
import pandas as pd
import package.get_NEM_data as gd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sep = ['\\','/'][sys.platform=='linux']
PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

font_size = 14
regions = ['SA','BGE']
years = [2010,2019]
Fig = make_subplots(rows=2,cols=2,
#                     subplot_titles=['%s-%d'%(x,y) for x,y in zip(2*regions,np.repeat(years,2))],
#                     shared_xaxes=True, 
                    vertical_spacing=0.27,
                    horizontal_spacing=0.15
                   )
bins=60
Opacity=1
for r,Region in enumerate(regions):
    for i,Year in enumerate(years):
        if Region in NEM:
                price_spot=gd.load_rrp_cal_stamped(Year, Region)
                price_spot['year']=Year
                price_spot.spot_price = price_spot[['spot_price','year']].apply(
                    lambda x: gd.convert_curr(*x ),axis=1)
                price_spot = price_spot.drop('year', axis=1)
                c = price_spot['spot_price']
                c_trunc= c[(c<300)&(c>-50)]

                Fig.add_trace(go.Histogram(x=c_trunc, nbinsx=bins, histnorm='probability',
                                          marker=dict(color='tomato', line=dict(width=0.75, color='black')),
                                          showlegend=False)
                                           ,row=i+1, col=r+1)
                
        elif Region in PJM:
#             price_spot=gd.load_lmp_zone_year(Region, Year, lmp_type='rt').reset_index()
#             price_spot.columns = [ 'date_time', 'spot_price']
#             price_spot['region'] = Region
#             c = price_spot['spot_price']
#             c_trunc= c[(c<300)&(c>-50)]
#             Fig.add_trace(go.Histogram(x=c_trunc, nbinsx=bins, histnorm='probability',
#                                        name='Real time',
#                                       marker=dict(opacity=Opacity, color='red',
#                                                   line=dict(width=0.75, color='black')),
#                                       showlegend=[True,False][i==1],
#                                       )
#                                        ,row=i+1, col=r+1)
            
            price_spot=gd.load_lmp_zone_year(Region, Year, lmp_type='da').reset_index()
            price_spot.columns = [ 'date_time', 'spot_price']
            price_spot['region'] = Region
    
            c = price_spot['spot_price']
            c_trunc= c[(c<300)&(c>-50)]
            Fig.add_trace(go.Histogram(x=c_trunc, histnorm='probability',
                                        xbins=dict(start=-50, end=300,size=10),
                                       name='Day ahead',
                                      marker=dict(opacity=Opacity, color='tomato',
                                                  line=dict(width=0.75, color='black')),
                                      showlegend=[False,False][i==1],
                                      )
                                       ,row=i+1, col=r+1)
        
        
        

for i in [1,2]:
    for j in [1,2]:
        Fig.update_xaxes({
#                           'title':['Spot price [$/MWh]',''][i==1],
                          'tickmode': 'linear',
                          'tick0': -200,
                          'dtick': 50,
                          'range':[-50,300],
                          'ticks':'outside',
                          'tickangle':90,
                          'tickwidth':2,
                          'ticklen': 10,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        Fig.update_yaxes({
#                           'title':'Probability',
                          'ticks':'outside',
                          'tickwidth':2,
                          'ticklen': 10,
                          'range':[0,0.5],
                          'nticks':8,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)

Fig.update_layout(plot_bgcolor='white',
                  width=800, height=550,
                  margin=dict(l=10, r=10, t=20, b=10),
                  font=dict(size=28),
                  barmode='overlay',
#                   legend=dict(
#                               orientation="h",
#                               yanchor="top",
#                               y=0.95,
#                               xanchor="center",
#                               x=0.85,
#                               )
                  )


savedir = paperdir + 'Pictures%sRRP annual%s\Histograms%s'%(sep,sep,sep)
# Fig.write_image(savedir + 'RRP_hist_%s_nature.svg'%(" ".join(regions)))
Fig.show()

# Plot SV vs RTE bar plots for SA and BGE

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import package.sql_manager as sm
import package.get_NEM_data as gd
from matplotlib.ticker import ScalarFormatter
from plotly.subplots import make_subplots
import plotly.graph_objects as go


sep = ['\\','/'][sys.platform=='linux']

PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

RTEs = np.arange(100,20,-10)
Colors = ['violet',
         'gold',
         'mediumslateblue',
         'orangered',
         'mediumseagreen',
         'magenta',
         'orange',
         'skyblue']

regions = ['SA', 'BGE']
years = np.arange(2010,2020,1)

font_size = 16
Fig = make_subplots(rows=1, cols=2, subplot_titles=(regions))


for R,Region in enumerate(regions):
    if Region in NEM:
        db = 'storage_value_PF_simple_%s.db'%(Region)
        reg_type = 'state'
        table = 'storage_value'
        cols = [x[0] for x in sm.list_columns(db,table)]
        Storage = sm.get_data(cols, table, db)
        Storage['date_time']=Storage['date_time'].astype(int)
    if Region in PJM:
        db = 'storage_value_arbi_capacity_PJM.db'
        reg_type = 'zone'
        table = 'storage_value'
        cols = [x[0] for x in sm.list_columns(db,table)]
        Storage = sm.get_data(cols, table, db)
        Storage['year']=Storage['year'].astype(int)
        Storage = Storage[Storage['arbi_type']=='DA+RT'].drop(
            ['arbi_type','SV_capa','SV_total'],axis=1)
        Storage.rename(columns={
            'year':'date_time',
            'SV_arbi':'SV',
            'pin_max':'Pin_max',
            'pout_max':'Pout_max'}, inplace=True)

    

    for (r,RTE) in enumerate(RTEs):
        data = pd.DataFrame(columns=['year', 'SV'])
        SV = Storage[(Storage[reg_type]==Region)&
                      (Storage['rte']==RTE) &
                      (Storage['sh']==10) &
                      (Storage['loss']==0)&
                      (Storage['Pin_max']==50)&
                      (Storage['Pout_max']==50)]
        if reg_type=='state':
                SV=SV[SV['cap']==20000]
#                 SV['SV']= SV[['SV','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)

        SV = SV.sort_values('date_time', axis=0)
        Fig.add_trace(go.Bar(x=SV['date_time'],
                     y=SV['SV']/(50),
                     name='%d%%'%(RTEs[r]),
                     marker_color=Colors[r],
                     marker_line_color='black',
                     width=0.5,
                     legendgroup='%d'%(RTE),
                     showlegend = [False,True][R==1]),
                     row=1, col=R+1)
        
for i in [1,2]:
    Fig.update_xaxes({'title':'Calendar year',
                      'tickmode': 'linear',
                      'tick0': 0,
                      'dtick': 1,
                      'ticks':'outside',
                      'tickangle':30,
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)
    Fig.update_yaxes({'title':['$/MW-year',''][i==2],
                      'ticks':'outside',
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)

Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='v',
                  legend=dict(x=1.01, y=0),
                  legend_title_text = '         RTE',
                  width=900, height=350,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')
Fig.show()
paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1"
savedir = paperdir + '%sPictures%sStorage value vs RTE%s'%(sep,sep,sep)
# plt.savefig(savedir + 'SV_RTE_%s.png'%(" ".join(regions)),  dpi=600, bbox_inches = "tight")

# Fig.write_image(savedir + 'SV_RTE_%s.png'%(" ".join(regions)))

# Plot the line graph of SV vs RTE

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import package.sql_manager as sm
import package.get_NEM_data as gd
from matplotlib.ticker import ScalarFormatter
from plotly.subplots import make_subplots
import plotly.graph_objects as go



sep = ['\\','/'][sys.platform=='linux']

PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

RTEs = np.arange(100,20,-10)
Colors = [
         'tomato',
         'deepskyblue']

regions = ['SA', 'BGE']
years = np.arange(2010,2020,1)

font_size = 16
Fig = make_subplots(rows=1, cols=1)

Data = pd.DataFrame(columns=['region', 'rte', 'sv'])

for R,Region in enumerate(regions):
    if Region in NEM:
        db = 'storage_value_PF_simple_%s.db'%(Region)
        reg_type = 'state'
        table = 'storage_value'
        cols = [x[0] for x in sm.list_columns(db,table)]
        Storage = sm.get_data(cols, table, db)
        Storage.date_time=Storage.date_time.astype(int)
    if Region in PJM:
        db = 'storage_value_arbi_capacity_PJM.db'
        reg_type = 'zone'
        table = 'storage_value'
        cols = [x[0] for x in sm.list_columns(db,table)]
        Storage = sm.get_data(cols, table, db)
        Storage['year']=Storage.year.astype(int)
        Storage = Storage[Storage.arbi_type=='DA+RT'].drop(
            ['arbi_type','SV_capa','SV_total'],axis=1)
        Storage.rename(columns={
            'year':'date_time',
            'SV_arbi':'SV',
            'pin_max':'Pin_max',
            'pout_max':'Pout_max'}, inplace=True)

    for (r,RTE) in enumerate(RTEs):
        data = pd.DataFrame(columns=['year', 'SV'])
        SV = Storage[(Storage[reg_type]==Region)&
                      (Storage.rte==RTE) &
                      (Storage.sh==10) &
                      (Storage.loss==0)&
                      (Storage.Pin_max==50)&
                      (Storage.Pout_max==50)]
        if reg_type=='state':
                SV=SV[SV.cap==20000]
                SV['SV']= SV[['SV','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)

        SV = SV.sort_values('date_time', axis=0)
        Data = Data.append({'region': Region,
                            'rte': RTE,
                            'sv': (SV.SV/50).tolist()},ignore_index=True)
        
Data.sort_values(['region','rte'])


def MEAN(x):
    return(np.mean(x))
def STD(x):
    return(np.std(x))
Data['sv_mean'] = Data[['sv']].apply(lambda x: MEAN(*x),axis=1)
Data['sv_std'] = Data[['sv']].apply(lambda x: STD(*x),axis=1)

for R, Region in enumerate(regions):
    DATA = Data[Data.region==Region]
    Fig.add_trace(go.Scatter(x=DATA.rte,
                             y=DATA.sv_mean,
                             name='%s'%(Region),
                             error_y=dict(type='data', array=(DATA.sv_std).tolist()),
                             line=dict(color=Colors[R]),
                             marker=dict(color=Colors[R], line_width=1, size=10),
                            ),
                             row=1, col=1)
        
for i in [1]:
    Fig.update_xaxes({'title':'RTE [%]',
                      'tickmode': 'linear',
                      'dtick': 10,
                      'ticks':'outside',
                      'tickangle':0,
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)
    Fig.update_yaxes({'title': ['Mean SV [$/MW-year]',''][i==2],
                      'ticks':'outside',
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)

Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='v',
                  legend=dict(x=0.05, y=0.8,xanchor='left',
                              yanchor='bottom',
                              bordercolor='black',
                              borderwidth=1),
                  width=500, height=350,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')
Fig.show()
paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1"
savedir = paperdir + '%sPictures%sStorage value vs RTE%s'%(sep,sep,sep)
# Fig.write_image(savedir + 'SV_RTE_line_hart_%s_test.png'%(" ".join(regions)))

# Plot REV and EXP for SA and BGE (Joe's version)

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import projdirs
import package.sql_manager as sm
import package.get_NEM_data as gd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pdb
sep = ['\\','/'][sys.platform=='linux']

PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']



Fig = make_subplots(rows=2,cols=2,
                    subplot_titles=['%s (RTE=%d%%)'%(x,y) for x,y in zip(2*['SA','BGE'],np.repeat([40,90],2))],
                    shared_xaxes=True,
#                     shared_yaxes=True,
                    vertical_spacing=0.1,
                    horizontal_spacing=0.1
                   )
colors = ['tomato','royalblue','hotpink','lightseagreen','deepskyblue','firebrick']
res=5

years_group = [[2010],[2019]]        
rtes = [40,90]



Region = 'SA'
prices = [10**x for x in np.arange(1,4.3,0.02)]
db = 'storage_value_perfect_%s.db'%(Region)
data = sm.get_data('N/A','N/A', db)
data.date_time = pd.to_datetime(data.date_time)
data['year']=data.date_time.dt.year
for r,RTE in enumerate(rtes):
    c=0
    REV_cdf = pd.DataFrame(columns=['years','price','rev_cum','exp_cum'])
    for y, years in enumerate(years_group):
        Data = data[(data.date_time.dt.year.isin(years) )&
                    (data.rte==RTE)].copy()
        
        Data['RRP']= Data[['RRP','year']].apply(lambda x: gd.convert_curr(*x ),axis=1)
        Data['rev'] = 0.5*Data.Pout * Data.RRP
        Data['exp'] = 0.5*Data.Pin * Data.RRP
     
        for price in prices:
            rev = (Data[(Data.RRP<=price)&
                           (Data.rev>0)]['rev'].sum()-\
                      Data[(Data.RRP<=price)&
                           (Data.exp<0)]['exp'].sum())/len(years)

            exp = (Data[(Data.RRP<=price)&
                           (Data.exp>0)]['exp'].sum()-\
                      Data[(Data.RRP<=price)&
                           (Data.rev<0)]['rev'].sum())/len(years)

            REV_cdf = REV_cdf.append({'years':str(years),
                                      'price':price,
                                      'rev_cum':rev,
                                      'exp_cum':exp
                                      },ignore_index=True)

        normaliser = 50
    
    for years in years_group:
        REV_cdf_year = REV_cdf[REV_cdf.years==str(years)]
        REV_cdf_normal = pd.DataFrame()
        REV_cdf_normal["price"] = REV_cdf_year.price
        
        col = REV_cdf_year.columns[2]
        REV_cdf_normal[col] = REV_cdf_year[col]/normaliser
        Name = col.replace('_cum','').upper()+\
                ' (%s)'%(  (',').join([str(x) for x  in years]).replace(',20','-') )
        Fig.add_trace(go.Scatter(x=REV_cdf_normal.price,
                                 y=REV_cdf_normal[col],
                                 name=Name,
                                 line=dict(color=colors[c]),
                                 showlegend=[True,False][r==1],
                                 legendgroup='REV',
                                 
                                  )
                               ,row=r+1, col=1)
        
        col = REV_cdf_year.columns[3]
        REV_cdf_normal[col] = REV_cdf_year[col]/normaliser
        Name = col.replace('_cum','').upper()+\
                ' (%s)'%(  (',').join([str(x) for x  in years]).replace(',20','-') )
        Fig.add_trace(go.Scatter(x=REV_cdf_normal.price,
                                 y=REV_cdf_normal[col],
                                 name=Name,
                                 line=dict(color=colors[c], dash='dash'),
                                 showlegend=[True,False][r==1],
                                 legendgroup='EXP'
                                  )
                               ,row=r+1, col=1)
        
        c+=1
        
 
            
Region = 'BGE'
prices = [10**x for x in np.arange(1,3.5,0.02)]
db = 'storage_value_perfect_%s.db'%(Region)
data = sm.get_data('N/A','N/A', db)
data.date_time = pd.to_datetime(data.date_time)
for r,RTE in enumerate(rtes):
    c=2
    REV_cdf = pd.DataFrame(columns=['years','price','rev_rt_cum','rev_da_cum'])
    for y, years in enumerate(years_group):
        Data = data[(data.date_time.dt.year.isin(years) )&
                    (data.rte==RTE)].copy()
        
        
        Data['exp_da'] = Data.Pin_da * Data.lmp_da
        Data['exp_rt'] = Data.Pin_rt * Data.lmp_rt
        Data['rev_da'] = Data.Pout_da * Data.lmp_da
        Data['rev_rt'] = Data.Pout_rt * Data.lmp_rt

        for price in prices:
            rev_rt = (Data[(Data.lmp_rt<=price)&
                           (Data.rev_rt>0)]['rev_rt'].sum()-\
                      Data[(Data.lmp_rt<=price)&
                           (Data.exp_rt<0)]['exp_rt'].sum())/len(years)

            rev_da = (Data[(Data.lmp_da<=price)&
                           (Data.rev_da>0)]['rev_da'].sum()-\
                      Data[(Data.lmp_da<=price)&
                           (Data.exp_da<0)]['exp_da'].sum())/len(years)

            exp_rt = (Data[(Data.lmp_rt<=price)&
                           (Data.exp_rt>0)]['exp_rt'].sum()-\
                      Data[(Data.lmp_rt<=price)&
                           (Data.rev_rt<0)]['rev_rt'].sum())/len(years)

            exp_da = (Data[(Data.lmp_da<=price)&
                           (Data.exp_da>0)]['exp_da'].sum()-\
                      Data[(Data.lmp_da<=price)&
                           (Data.rev_da<0)]['rev_da'].sum())/len(years)

            REV_cdf = REV_cdf.append({'years': str(years),
                                      'price':price,
                                      'rev_rt_cum':rev_rt,
                                      'rev_da_cum':rev_da,
                                      'exp_da_cum':exp_da,
                                      'exp_rt_cum':exp_rt
                                                   },ignore_index=True)
        normaliser = 50
    for years in years_group:
        REV_cdf_year = REV_cdf[REV_cdf.years==str(years)]
        REV_cdf_normal = pd.DataFrame()
        REV_cdf_normal["price"] = REV_cdf_year.price
        for k in [2,3]:
            col = REV_cdf_year.columns[k]
            REV_cdf_normal[col] = REV_cdf_year[col]/normaliser
            Name = col.replace('_cum','').replace('_',' (').upper()+\
                         ' %s)'%(  (',').join([str(x) for x  in years]).replace(',20','-') )
            Fig.add_trace(go.Scatter(x=REV_cdf_normal.price,
                                     y=REV_cdf_normal[col],
                                     mode='lines',
                                     name=Name,
                                     line=dict(color=colors[c]),
                                     showlegend=[True,False][r==1],
                                     legendgroup='REV'
                                      )
                                   ,row=r+1, col=2)
            col = REV_cdf_year.columns[k+2]
            REV_cdf_normal[col] = REV_cdf_year[col]/normaliser
            Name = col.replace('_cum','').replace('_',' (').upper()+\
                         ' %s)'%(  (',').join([str(x) for x  in years]).replace(',20','-') )
            Fig.add_trace(go.Scatter(x=REV_cdf_normal.price,
                                     y=REV_cdf_normal[col],
                                     mode='lines',
                                     name=Name,
                                     line=dict(color=colors[c], dash='dash'),
                                     showlegend=[True,False][r==1],
                                     legendgroup='EXP'
                                      )
                                   ,row=r+1, col=2)
            c+=1

        
for i in [1,2]:
    for j in [1,2]:      
        Fig.update_xaxes({'title':['Spot price [$/MWh]',''][i==1],
                          'range':[1,[np.log10(3000), np.log10(20000)][j==1]],
                          'type':'log',
                          'tickvals':[[10,50,100,300,500,700,1000,2000],
                                      [10,40,60,100,170,300,1000,2000,5000,10000, 15000]
                                     ][j==1],
                          'ticks':'outside',
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        Fig.update_yaxes({'title':['Cum. turnover [$/MW-Year]',''][j==2],
                          'ticks':'outside',
                          'nticks':10,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)

Fig.update_layout(plot_bgcolor='white',
                  width=900, height=700,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay',
                  legend=dict(
                              orientation="h",
                              yanchor="bottom",
                              y=1.05,
                              xanchor="center",
                              x=0.5,
                              )
                  )

paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1"
savedir = paperdir + '%sPictures%sRevenue distribution%s'%(sep,sep,sep)
# Fig.write_image(savedir + 'Rev_Exp_distribution_%s.svg'%("_".join(['SA','BGE'])))
# Fig.write_image(savedir + 'Rev_Exp_distribution_%s.png'%("_".join(['SA','BGE'])))
Fig.show()

# AOI for SA and BGE

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Fig = make_subplots(rows=1, cols=2, subplot_titles=(regions))
Colors = ['violet',
          'gold',
          'mediumslateblue',
          'orangered',
          'mediumseagreen',
          'magenta',
          'orange',
          'skyblue']
for R,Region in enumerate(regions):
    for r,RTE in enumerate(RTEs):
        DATA = SPIKES[(SPIKES['rte']==RTE)&
                      (SPIKES['region']==Region)]
        Fig.add_trace(go.Scatter(x=DATA['year'],
                                 y=DATA['spikes_vol']/normaliser,
                                 line = dict(width=1),
                                 name='%d%%'%(RTEs[r]),
                                 marker_color=Colors[r],
                                 marker_line_color='black',
                                 legendgroup='%d'%(RTE),
                                 showlegend = [False,True][R==1]),
                                 row=1, col=R+1)


for i in [1,2]:
    Fig.update_xaxes({'title':'Calendar year',
                      'tickmode': 'linear',
                      'tick0': 0,
                      'dtick': 1,
                      'ticks':'outside',
                      'tickangle':30,
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)
    Fig.update_yaxes({'title':['AOI (normalised)',''][i==2],
                      'ticks':'outside',
                      'tick0': 0,
                      'dtick': 0.1,
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)

Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='v',
                  legend=dict(x=1.01, y=0),
                  legend_title_text = 'RTE',
                  width=900, height=350,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')
Fig.update_traces(marker=dict(size=6,
                              line=dict(width=1,
                                        color='black')))
Fig.show()
paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\\"
savedir = paperdir + 'Pictures%sstorage value vs RTE%sspike volume%s'%(sep,sep,sep)
Fig.write_image(savedir + 'spike_volume_%s.png'%(" ".join(regions)))


# Plot the SV including the arbitrage and capacity payments

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

from projdirs import paperdir
import numpy as np
import pandas as pd
import package.get_NEM_data as gd
import package.sql_manager as sm
from plotly.subplots import make_subplots
import plotly.graph_objects as go

sep = ['\\','/'][sys.platform=='linux']

NEM = ['SA','VIC','NSW','QLD','TAS']
PJM = ['DPL', 'BGE']

regions = ['SA','BGE']
RTEs = np.arange(90,20,-20)
averagings = ['annually', 'quarterly']
averaging = averagings[0]

Fig = make_subplots(rows=1, cols=2, subplot_titles=(regions))
Colors = ['violet','gold','mediumslateblue','orangered','mediumseagreen',
         'magenta','orange','skyblue']

for R,Region in enumerate(regions):
    
    if Region in PJM:
        db = 'storage_value_arbi_capacity_PJM.db'
        data = sm.get_data('N/A', 'N/A', db)
        for r,RTE in enumerate(RTEs):
            DATA = data[(data.arbi_type=='DA+RT')&
                        (data.zone==Region)&
                        (data.pout_max==50)&
                        (data.sh==10)&
                        (data.rte==RTE)]
            Fig.add_trace(go.Bar(x=DATA.year,
                                 y=DATA.SV_total/(50),
                                 name='%d%%'%(RTEs[r]),
                                 marker_color=Colors[r],
                                 marker_line_color='black',
                                 width=0.5,
                                 legendgroup='%d'%(RTE),
                                 showlegend = [False,True][R==1]),
                                 row=1, col=R+1)
    elif Region in NEM:
        db = 'SV_NEM_cap_contract.db'
        data = sm.get_data('N/A', 'N/A', db)
        data['SV_total']= data[['SV_total','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
        for r,RTE in enumerate(RTEs):
            DATA = data[(data.state=='SA')&
                        (data.Pout_max==50)&
                        (data.sh==10)&
                        (data.cap_sampling==averaging)&
                        (data.rte==RTE)]
            
            Fig.add_trace(go.Bar(x=DATA.date_time,
                                 y=DATA.SV_total/(50),
                                 name='%d%%'%(RTEs[r]),
                                 marker_color=Colors[r],
                                 marker_line_color='black',
                                 width=0.5,
                                 legendgroup='%d'%(RTE),
                                 showlegend = [False,True][R==1]),
                                 row=1, col=R+1)
# Fix the layout
for i in [1,2]:
    Fig.update_xaxes({
#                       'title':'Calendar year',
                      'tickmode': 'linear',
                      'tick0': 0,
                      'dtick': 1,
                      'ticks':'outside',
                      'tickangle':30,
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)
    Fig.update_yaxes({'title':['$/MW-year',''][i==2],
                      'ticks':'outside',
                      'range':[0,360000],
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)

Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='v',
                  legend=dict(x=0.9, y=1),
                  legend_title_text = '         RTE',
                  width=900, height=350,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')
Fig.show()
savedir = paperdir + 'Pictures%sStorage value vs RTE and capacity%s'%(sep,sep)
# Fig.write_image(savedir + 'SV_RTE_%s_Nature.svg'%(" ".join(regions)))

# comparing the SV with and without capacity payments

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

from projdirs import paperdir
import numpy as np
import pandas as pd
import package.get_NEM_data as gd
import package.sql_manager as sm
from plotly.subplots import make_subplots
import plotly.graph_objects as go

sep = ['\\','/'][sys.platform=='linux']

NEM = ['SA','VIC','NSW','QLD','TAS']
PJM = ['DPL', 'BGE']

regions = ['SA','BGE']
RTEs = np.arange(90,30,-50)
averagings = ['annually', 'quarterly']
averaging = averagings[0]

Fig = make_subplots(rows=1, cols=2, subplot_titles=(regions), shared_yaxes=True, horizontal_spacing=0.05)
Colors1 = ['skyblue','pink']
Colors2 = ['dodgerblue','tomato']

Offset = 0.3
for R,Region in enumerate(regions):
    
    if Region in NEM:
        db = 'storage_value_PF_simple_%s.db'%(Region)
        reg_type = 'state'
        Storage = sm.get_data('N/A', 'N/A', db)
        Storage['date_time']=Storage['date_time'].astype(int)
    if Region in PJM:
        db = 'storage_value_arbi_capacity_PJM.db'
        reg_type = 'zone'
        Storage = sm.get_data('N/A', 'N/A', db)
        Storage['year']=Storage['year'].astype(int)
        Storage = Storage[Storage['arbi_type']=='DA+RT'].drop(
            ['arbi_type','SV_capa','SV_total'],axis=1)
        Storage.rename(columns={
                                'year':'date_time',
                                'SV_arbi':'SV',
                                'pin_max':'Pin_max',
                                'pout_max':'Pout_max'}, inplace=True)

    for (r,RTE) in enumerate(RTEs):
        data = pd.DataFrame(columns=['year', 'SV'])
        SV = Storage[(Storage[reg_type]==Region)&
                      (Storage['rte']==RTE) &
                      (Storage['sh']==10) &
                      (Storage['loss']==0)&
                      (Storage['Pin_max']==50)&
                      (Storage['Pout_max']==50)]
        if reg_type=='state':
                SV=SV[SV['cap']==20000]
                SV['SV']= SV[['SV','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)

        SV = SV.sort_values('date_time', axis=0)
        Fig.add_trace(go.Bar(x=SV['date_time'],
                     y=SV['SV']/(50),
                     name='%d%%-E'%(RTEs[r]),
                     marker_color=Colors1[r],
                     marker_line_color='black',
#                      opacity=0.5,
                     width=0.3,
                     offset=-Offset,
                     legendgroup='E',
                     showlegend = [False,True][R==1]),
                     row=1, col=R+1)
        
        
    
    
    
    
    if Region in PJM:
        db = 'storage_value_arbi_capacity_PJM.db'
        data = sm.get_data('N/A', 'N/A', db)
        for r,RTE in enumerate(RTEs):
            DATA = data[(data.arbi_type=='DA+RT')&
                        (data.zone==Region)&
                        (data.pout_max==50)&
                        (data.sh==10)&
                        (data.rte==RTE)]
            Fig.add_trace(go.Bar(x=DATA.year,
                                 y=DATA.SV_total/(50),
                                 name='%d%%-E&C'%(RTEs[r]),
                                 marker_color=Colors2[r],
                                 marker_line_color='black',
                                 width=0.3,
                                 offset=0,
                                 legendgroup='E&C',
                                 showlegend = [False,True][R==1]),
                                 row=1, col=R+1)
    elif Region in NEM:
        db = 'SV_NEM_cap_contract.db'
        data = sm.get_data('N/A', 'N/A', db)
        data['SV_total']= data[['SV_total','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
        for r,RTE in enumerate(RTEs):
            DATA = data[(data.state==Region)&
                        (data.Pout_max==50)&
                        (data.cap_sampling==averaging)&
                        (data.rte==RTE)]
            
            Fig.add_trace(go.Bar(x=DATA.date_time,
                                 y=DATA.SV_total/(50),
                                 name='%d%%-E&C'%(RTEs[r]),
                                 marker_color=Colors2[r],
                                 marker_line_color='black',
                                 width=0.3,
                                 offset=0,
                                 legendgroup='E&C',
                                 showlegend = [False,True][R==1]),
                                 row=1, col=R+1)
# Fix the layout
for i in [1,2]:
    Fig.update_xaxes({'title':'Calendar year',
                      'tickmode': 'linear',
                      'tick0': 0,
                      'dtick': 1,
                      'ticks':'outside',
                      'tickangle':30,
                      'showgrid':False,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)
    Fig.update_yaxes({'title':['$/MW-year',''][i==2],
                      'ticks':'outside',
                      'range':[0,360000],
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)

Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='h',
                  legend=dict(x=0.5, y=1.01, xanchor='center', yanchor='bottom'),
#                   legend_title_text = 'RTE',
                  width=900, height=400,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')
Fig.show()
paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\\"
savedir = paperdir + 'Pictures%sStorage value vs RTE and capacity%s'%(sep,sep)
Fig.write_image(savedir + 'SV_RTE_%s_comparison.png'%(" ".join(regions)))

# Pearson R analysis on between the SV and STD

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

from projdirs import paperdir
from datetime import timedelta
import datetime
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm, dates
import pdb
import pandas as pd
import package.get_NEM_data as gd
import package.sql_manager as sm
from scipy.stats import pearsonr

sep = ['\\','/'][sys.platform=='linux']
savedir = paperdir + 'Pictures%sPearsonR%s'%(sep,sep)

states = ['SA', 'NSW']
years = np.arange(2010,2019,1)
RTEs = np.arange(30,110,10)
P = pd.DataFrame(columns=['cap','30%p','40%p','50%p','60%p'
                         ,'70%p','80%p','90%p','100%p'])
for Cap in [300, 1000, 2000, 4000, 8000, 14500]:
    print(Cap)
    P = P.append({'cap':Cap},ignore_index=True)
    data = pd.DataFrame(columns=['state','year', 'mean','std',
                             '30%', '40%', '50%', '60%',
                             '70%', '80%', '90%', '100%'])

    for s,State in enumerate(states):
        table = 'storage_value'
        db = 'storage_value_PF_simple_%s.db'%(State)
        cols = [x[0] for x in sm.list_columns(db,table)]
        Storage = sm.get_data(cols, table, db)
        Storage['date_time']=Storage['date_time'].astype(int)

        for i,Year in enumerate(years):
            price_spot=pd.DataFrame(gd.load_rrp_cal_stamped(Year, State))
            price_spot.columns = ['region', 'date_time', 'spot_price'] # define the data columns 
            c = price_spot['spot_price']
            c_trunc= c[c<Cap]
            data = data.append({'state':State,
                                 'year':Year,
                                  'mean':c_trunc.mean(),
                                 'std':c_trunc.std()}, ignore_index=True)
            for (r,RTE) in enumerate(RTEs):
                SV = Storage[(Storage['date_time']==Year) &
                              (Storage['cap']==14500) &
                              (Storage['rte']==RTE) &
                              (Storage['sh']==10) &
                              (Storage['loss']==0)]['SV'].values[0]
                ind = data.index[(data['state']==State)&
                                 (data['year']==Year)]
                data.loc[ind,'%d%%'%(RTE)]=SV

    for RTE in RTEs:
        ind_p = P.index[(P['cap']==Cap)]
        P.loc[ind_p,'%d%%p'%(RTE)]=pearsonr(
            data['%d%%'%(RTE)].tolist(), data['std'].tolist())[0]


# plt.rcParams['text.usetex'] = True
font_size = 14
Colors = ['k','r', 'g', 'm', 'c', 'b' ]
fig, ax = plt.subplots(figsize=(8,6),nrows=1, ncols=1)
for c,Cap in enumerate(P['cap'].tolist()):
    ax.plot(np.arange(30,110,10),P[P['cap']==Cap].values[0][1:],
            label='%d'%(Cap), marker='o', 
            linestyle=['--','-'][(c==0) or (c==len(P['cap'].tolist())-1)], 
            color=Colors[c])

lg = plt.legend(title='Truncated at MWh', loc='upper center',
           bbox_to_anchor=(0.5, 1.01), ncol=len(RTEs),
           fontsize=font_size, edgecolor='k',
           handletextpad=0.1, columnspacing =1.37,
           bbox_transform = plt.gcf().transFigure)
                
lg.get_title().set_fontsize(font_size)
ax.grid(which='major', axis='both')
ax.set_axisbelow(True)
plt.rc('ytick', labelsize=font_size)
plt.rc('xtick', labelsize=font_size)
ax.set_xlabel('RTE', fontsize=font_size)
ax.set_ylabel('R', fontsize=font_size)
# plt.savefig(savedir + 'R_RTE.png', format='png', dpi=600)
plt.show()

# Create SV vs SH 

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import package.sql_manager as sm
import pdb
import matplotlib.gridspec as gridspec
from matplotlib.ticker import ScalarFormatter


sep = ['\\','/'][sys.platform=='linux']



States = ['SA', 'NSW']
RTEs = np.arange(100,20,-10)
SHours = np.arange(18,0,-1)
loss = 0
cap = 14500
Colors = ['k', 'gray','r', 'g', 'b', 'm', 'y', 'c', ]
plt.close("all")
font_size = 16
years = np.arange(2017,2020,1)
fig, axes = plt.subplots(figsize=(16,15),nrows=len(years), ncols=2, gridspec_kw = {'wspace':0.15, 'hspace':0.1})
ax = axes.ravel()
xfmt = ScalarFormatter()
xfmt.set_powerlimits((-3,3))

for s,state in enumerate(States):
    db = 'storage_value_PF_simple_%s.db'%(state)
    table = 'storage_value'
    cols = [x[0] for x in sm.list_columns(db,table)]
    data = sm.get_data(cols,table, db)
    for i,year in enumerate(years):
        for j,RTE in enumerate(RTEs):
            SV = data[(data['date_time']==year) &
                      (data['cap']==cap) &
                      (data['rte']==RTE) &
                      (data['loss']==loss)&
                      (data['Pin_max']==50)&
                      (data['Pout_max']==50)]

            SV = SV.sort_values('sh', axis=0)
            ax[(2*i)+s].plot(SV['sh'], SV['SV']/(50), color=Colors[j],
             label= '%d'%(RTE), marker='o')
        ax[(2*i)+s].set_title('Year=%d'
                              % (year), fontsize=font_size,
                              position=[0.2,0.85],
                              bbox=dict(facecolor='w', edgecolor='k'))
        if s==0:
            ax[(2*i)+s].set_ylabel('SV [A$/MW-year]', fontsize=font_size)
        
        if year==years[-1]:
            ax[(2*i)+s].set_xlabel('Storage hours [hrs]', fontsize=font_size)
        else:
            ax[(2*i)+s].tick_params(labelbottom=False)
        
        if year==years[0]:
            ax[(2*i)+s].text(0.5,1.05,'%s'%(state),
                  fontsize=font_size, transform=ax[(2*i)+s].transAxes, ha='center')
        
        ax[(2*i)+s].grid(which='major', axis='both')
        ax[(2*i)+s].set_axisbelow(True)
#         ax[(2*i)+s].set_xticks(SHours[::2])
        ax[(2*i)+s].yaxis.set_major_formatter(xfmt)
        ax[(2*i)+s].set_xscale('log')

plt.rc('ytick', labelsize=font_size)

lg = plt.legend(title='RTE', loc='upper center',
               bbox_to_anchor=(0.5, 0.99), ncol=len(RTEs),
               fontsize=font_size, edgecolor='k',
               handletextpad=0.1, columnspacing =1.37,
               bbox_transform = plt.gcf().transFigure)
                
lg.get_title().set_fontsize(font_size)

paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\\"
savedir = paperdir + 'Pictures%sStorage value vs SH_line chart%s'%(sep,sep)
plt.savefig(savedir + 'SV_SH.png', format='png', dpi=600)
plt.show()

# Plot SV vs SH for SA and BGE

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm
import package.get_NEM_data as gd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

sep = ['\\','/'][sys.platform=='linux']
PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

regions = ['SA', 'BGE']
RTEs = np.arange(90,20,-30)
SHours = np.arange(18,0,-1)
loss = 0
cap = 14500
Colors = ['violet','gold','mediumslateblue','orangered','mediumseagreen',
         'magenta','orange','skyblue']

years = [2014,2016]
Fig = make_subplots(rows=2, cols=2,
                    subplot_titles=(['%s (%d)'%(x, y) for x,y in zip(regions*2,np.repeat(years,2))]),
                    vertical_spacing=0.1)
for R,Region in enumerate(regions):
    if Region in NEM:
        db = 'storage_value_PF_simple_%s.db'%(Region)
        reg_type = 'state'
    if Region in PJM:
        db = 'storage_value_PF_simple_PJM.db'
        reg_type = 'zone'
        
    table = 'storage_value'
    cols = [x[0] for x in sm.list_columns(db,table)]
    data = sm.get_data(cols,table, db)
    for i,year in enumerate(years):
        for r,RTE in enumerate(RTEs):
            SV = data[(data['date_time']==year) &
                      (data[reg_type]==Region)&
                      (data['rte']==RTE) &
                      (data['loss']==loss)&
                      (data['Pin_max']==50)&
                      (data['Pout_max']==50)&
                      (data['sh']<19)]
            if reg_type=='state':
                SV=SV[SV['cap']==cap]
                SV['SV']= SV[['SV','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)

            SV = SV.sort_values('sh', axis=0)
            
            Fig.add_trace(go.Scatter(x=SV['sh'],
                                 y=SV.SV/SV[SV.sh==1]['SV'].values[0],
                                 line = dict(width=1),
                                 name='%d%%'%(RTEs[r]),
                                 marker=dict(color=Colors[r], line_width=1, size=7),
                                 legendgroup='%d'%(RTE),
                                 showlegend = [False,True][(R==1)&(i==1)]),
                                 row=i+1, col=R+1)

for i in [1,2]:
    for j in [1,2]:
        Fig.update_xaxes({'title':['','SH [Hours]'][i==2],
                          'tickmode': 'linear',
                          'tick0': 0,
                          'dtick': 2,
                          'showticklabels':[False,True][i==2],
                          'ticks':'outside',
                          'tickangle':0,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        Fig.update_yaxes({'title':[r'$\textrm{SV}/\textrm{SV}_{\textrm{sh}=1}$',''][j==2],
                          'ticks':'outside',
                          'nticks':14,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        
Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='h',
                  legend=dict(x=0.5, y=1.02,xanchor='center',yanchor='bottom'),
                  legend_title_text = 'RTE',
                  width=900, height=700,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')
savedir = paperdir + 'Pictures%sStorage value vs SH_line chart%s'%(sep,sep)
Fig.show()
# Fig.write_image(savedir + 'SV_SH_%s.svg'%("-".join(regions)))


# New Graph

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm
import package.get_NEM_data as gd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pdb

sep = ['\\','/'][sys.platform=='linux']
PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

def MEAN(x):
    return(np.mean(x))
def STD(x):
    return(np.std(x))
            
regions = ['SA', 'BGE']
RTEs = np.arange(90,20,-30)
SHours = np.arange(18,0,-1)
loss = 0
cap = 14500
Colors = ['violet','gold','mediumslateblue','orangered','mediumseagreen',
         'magenta','orange','skyblue']

Fig = make_subplots(rows=2, cols=2,
#                     subplot_titles=(['%s (%d)'%(x, y) for x,y in zip(regions*2,np.repeat(years,2))]),
                    vertical_spacing=0.1)

for R,Region in enumerate(regions):
    i=1
    if Region in NEM:
        db = 'storage_value_PF_simple_%s.db'%(Region)
        reg_type = 'state'
    if Region in PJM:
        db = 'storage_value_PF_simple_PJM.db'
        reg_type = 'zone'
        
    table = 'storage_value'
    cols = [x[0] for x in sm.list_columns(db,table)]
    data = sm.get_data(cols,table, db)
    if Region in NEM:
        data=data[data.cap==cap]
#         data['SV']= data[['SV','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
        
    for years in (np.arange(2017,2019,1),np.arange(2010,2020,1)):
        
        for r,RTE in enumerate(RTEs):
            SV = data[(data.date_time.isin(years)) &
                      (data[reg_type]==Region)&
                      (data.rte==RTE) &
                      (data.loss==loss)&
                      (data.Pin_max==50)&
                      (data.Pout_max==50)&
                      (data.sh<19)]
#             pdb.set_trace()
            
                        
            
    
            SV = SV.sort_values('sh', axis=0)
            SV_group = SV.groupby('sh')
            Fig.add_trace(go.Scatter(x=SV_group['sh'].mean().tolist(),
                                     y=SV_group['SV'].mean()/SV[SV.sh==1]['SV'].mean(),
                                     error_y=dict(type='data', array=(
                                         SV_group['SV'].std()/SV[SV.sh==1]['SV'].mean()).tolist()),
#                                      line=dict(color=Colors[R]),
                                     marker=dict(color=Colors[r], line_width=1, size=5),
                                     legendgroup=str(RTE),
                                     name= str(RTE),
                                     showlegend=[False,True][(R==0)]
                                    ),
                                     row=i, col=R+1)
        i+=1

for i in [1,2]:
    for j in [1,2]:
        Fig.update_xaxes({'title':['','SH [Hours]'][i==2],
                          'tickmode': 'linear',
                          'tick0': 0,
                          'dtick': 2,
                          'showticklabels':[False,True][i==2],
                          'ticks':'outside',
                          'tickangle':0,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        Fig.update_yaxes({'title':['SV [$/MW-year]',''][j==2],
                          'ticks':'outside',
                          'nticks':14,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        
Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='v',
                  legend=dict(x=1.01, y=0),
                  legend_title_text = '         RTE',
                  width=900, height=600,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')
savedir = paperdir + 'Pictures%sStorage value vs SH_line chart%s'%(sep,sep)
Fig.show()
# Fig.write_image(savedir + 'SV_SH_%s.svg'%("-".join(regions)))


# Plot IRR_CPE_SH for various CPP

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import package.sql_manager as sm
import pdb
import matplotlib.gridspec as gridspec
import package.get_NEM_data as gd
# plt.rcParams['text.usetex'] = True


state = 'SA'
sh = np.arange(1,19,1)
db = 'storage_value_PF_simple_%s.db'%(state)
data = sm.get_data('N/A','N/A', db)
CPE_range = np.arange(1,500,1) # $/kWh

Pout_max = 50
Pin_max = 50

inputs = pd.DataFrame({'CPP':[70,100,1200,2000],
                       'RTE':[80,80,40,40],
                       'c0':[420,420,180,180]})

font_size = 14
fig, ax = plt.subplots(figsize=(16,12),nrows=2, ncols=2, gridspec_kw = {'wspace':0.1, 'hspace':0.1})
ax = ax.ravel()
import matplotlib.colors as colors
cmap = plt.get_cmap('jet')
new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n='jet', a=0, b=1),
        cmap(np.linspace(0.22, 0.88, 100)))


IRR = np.zeros( ( len(CPE_range), len(sh)))
for i in inputs.index.tolist():
    for s,SH in enumerate(sh):
        Pout_max = 50 #MWe
        Pin_max = 50 #MWe
        E_max = Pout_max/(inputs.iloc[i]['RTE']/100)*SH #MWhth
        SVs = data[(data['date_time']>2015)&
                   (data['sh']==SH)&
                   (data['Pout_max']==Pout_max)&
                   (data['Pin_max']==Pin_max)&
                   (data['rte']==inputs.iloc[i]['RTE'])&
                   (data['loss']==0)&
                   (data['cap']==14500)].sort_values('date_time',axis=0)
        SVs['SV_converted'] = SVs[['SV','date_time']].apply(lambda x: gd.convert_curr(x['SV'],x['date_time']),axis=1)
        for cpe,CPE in enumerate(CPE_range):
            CAPEX = Pout_max*1000*inputs.iloc[i]['CPP']+E_max*1000*CPE + Pout_max*1000*inputs.iloc[i]['c0']
            cashflows = np.insert(SVs.SV_converted.mean()*np.ones(20),0,-CAPEX)
            IRR[cpe,s] = round(np.irr(cashflows),4)
#             if (CPE==20) & (SH==10):
#                 P1= IRR[cpe,s]
#             if (CPE==20) & (SH==12):
#                 P2= IRR[cpe,s]

        X, Y = np.meshgrid(sh, CPE_range)
    L = np.linspace(0, [IRR.max(),2][IRR.max()>2], 10)
    im0 = ax[i].contourf(X,Y, IRR, 30, cmap=new_cmap, )
    im = ax[i].contour(X,Y, IRR, colors='b', levels=L, linestyles='-')
#     if cpp==0:
#         op = 2
#     elif cpp==1:
#         op=3
#     elif cpp==2:
#         op=7
#     elif cpp==3:
#         op=9
    
#     ax[cpp].plot([1,op],[12,12],linestyle='--',color='k')
#     ax[cpp].plot([op],[12],marker='o',color='k')
    
    ax[i].set_yscale('log')
    ax[i].set_xticks(np.arange(2,20,2))
    ax[i].clabel(im, colors = 'k', fmt = '$%2.3f$', fontsize=font_size-2)
    ax[i].grid(which='both', axis='both')
    if (i==0) or (i==2):
        ax[i].set_ylabel('CPE [$/kWh]', fontsize=font_size)
    else:
        ax[i].tick_params(labelleft=False)
    if i>1:
        ax[i].set_xlabel('SH [hr]', fontsize=font_size)
    else:
        ax[i].tick_params(labelbottom=False)
    ax[i].text(0.6,0.8,'CPP: %d $/kW \n RTE: %d%%'%(inputs.iloc[i]['CPP'],inputs.iloc[i]['RTE']),
              fontsize=font_size, transform=ax[i].transAxes,
               ha='left', bbox=dict(facecolor='w', edgecolor='k'))
plt.rc('ytick', labelsize=font_size)
plt.rc('xtick', labelsize=font_size)

sep = ['\\','/'][sys.platform=='linux']
savedir = paperdir + 'Pictures%sIRR_SH_RTE%s'%(sep,sep)
# plt.savefig(savedir + 'IRR_SH_CPE_%s_%d.png'%('SA',RTE), format='png', dpi=600, bbox_inches = "tight")
# plt.savefig(savedir + 'IRR_SH_CPE_%s_%d.svg'%('SA',RTE), bbox_inches = "tight")
plt.show()

# Alternative plot for IRR vs SH: This one uses cost figures for Li battery and molten salt storage from NREL

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import package.sql_manager as sm
import pdb
import matplotlib.gridspec as gridspec
import package.get_NEM_data as gd
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import matplotlib.colors as colors

Pout_max = 50
Pin_max = 50
RTEs = [40,50,80,90]
sh = np.arange(0.5,19.5,0.5)

state = 'SA'
db = 'storage_value_PF_simple_%s.db'%(state)
data = sm.get_data('N/A','N/A', db)
Data = data[(data.date_time>2015)&
            (data.Pout_max==Pout_max)&
            (data.Pin_max==Pin_max)&
            (data.loss==0)&
            (data.cap==14500)].sort_values(['date_time','sh'])

TEMP = Data[['date_time','rte','sh','SV']]
DATA = pd.DataFrame(columns=TEMP.columns)
for Year in TEMP.date_time.unique():
    for RTE in RTEs:
        temp1=TEMP[(TEMP.rte==RTE)&(TEMP.date_time==Year)].copy()
        temp2 = temp1.iloc[1:-1].copy()
        temp2.sh=temp2.sh+0.5
        temp2 = temp2.drop('SV',axis=1)
        
        temp = temp1.append(temp2,sort=False).sort_values(['date_time','rte','sh']).reset_index(drop=True).interpolate(method='polynomial',order=3)
        DATA = DATA.append(temp,ignore_index=True)


font_size = 14
fig, ax = plt.subplots(figsize=(16,12),nrows=2, ncols=2, gridspec_kw = {'wspace':0.1, 'hspace':0.1})
ax = ax.ravel()
cmap = plt.get_cmap('jet')
new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n='jet', a=0, b=1),
        cmap(np.linspace(0.22, 0.88, 100)))

for i,RTE in enumerate(RTEs):
    CPE_range = np.arange(1,[800,200][RTE in [40,50]],1) # $/kWh
    IRR = np.zeros( ( len(CPE_range), len(sh)))
    for s,SH in enumerate(sh):
        E_max = Pout_max/(RTE)*SH #MWhth
        SVs = DATA[(DATA['sh']==SH)&
                   (DATA['rte']==RTE)
                  ].sort_values('date_time',axis=0)
        SVs['SV_converted'] = SVs[['SV','date_time']].apply(lambda x: gd.convert_curr(x['SV'],x['date_time']),axis=1)
        for cpe,CPE in enumerate(CPE_range):
            if RTE in [80,90]:
                CAPEX = (CPE/(RTE/100)*Pout_max*1000*SH+5/6*17639695) #CPSH from NREL=18410296
            elif RTE in [40,50] :
                CAPEX = (CPE/(RTE/100)*Pout_max*1000*SH+50*1000*(1190+340)) #CPSH from NREL=18410296
            
            cashflows = np.insert(SVs.SV_converted.mean()*np.ones(20),0,-CAPEX)
            IRR[cpe,s] = 100*round(np.irr(cashflows),4)

        X, Y = np.meshgrid(sh, CPE_range)
    L = np.linspace(0, [IRR.max(),200][IRR.max()>200], 10)
    im0 = ax[i].contourf(X,Y, IRR, 30, cmap=new_cmap, )
    im = ax[i].contour(X,Y, IRR, colors='b', levels=L, linestyles='-')
    for j in np.arange(0,9,1):
        p = im.collections[j].get_paths()[0]
        v = p.vertices
        xy = v[np.argmax(v[:,1]),:]
        if np.max(v[:,1])<500:
            ax[i].plot(xy[0],xy[1],'o',markerfacecolor='white',
                       markeredgecolor='black', markersize=8)
            
    ax[i].set_yscale('log')
    ax[i].set_xticks(np.arange(2,20,2))
    ax[i].xaxis.set_minor_locator(MultipleLocator(1))
    ax[i].clabel(im, colors = 'k', fmt = ['%2.0f%%','%2.1f%%'][i in [0,1]], fontsize=font_size-1)
    ax[i].grid(which='both', axis='both')
    
    if i in [0,2]:
        ax[i].set_ylabel('CPE [$/kWh]', fontsize=font_size)
    else:
        ax[i].tick_params(labelleft=False)
    if i>1:
        ax[i].set_xlabel('SH [hr]', fontsize=font_size)
    else:
        ax[i].tick_params(labelbottom=False)
    ax[i].text(0.6,0.8,'RTE = %d%%'%(RTE),
              fontsize=font_size, transform=ax[i].transAxes,
               ha='left', bbox=dict(facecolor='w', edgecolor='k'))
plt.rc('ytick', labelsize=font_size)
plt.rc('xtick', labelsize=font_size)

sep = ['\\','/'][sys.platform=='linux']
savedir = paperdir + 'Pictures%sIRR_SH_RTE%s'%(sep,sep)
plt.savefig(savedir + 'IRR_SH_CPE_%s_%d.svg'%('SA',RTE), bbox_inches = "tight")
plt.show()

# Create IRR for CPE and CPP (SH=4)

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import matplotlib as mpl
import package.sql_manager as sm
import pdb
import matplotlib.gridspec as gridspec
from numpy import average as av
%matplotlib inline
plt.rcParams['text.usetex'] = True
sep = ['\\','/'][sys.platform=='linux']
savedir = paperdir + 'Pictures%sIRR_CPE_CPP%s'%(sep,sep)
def er(x,y):
    return(y-av([x,y]))

CPE_range = np.concatenate((np.arange(1,10,1), np.arange(10,1010,50)),axis=0  ) # $/kWh
CPP_range = np.concatenate( (np.arange(10,100,2), np.arange(100,6000,100)),axis=0 ) # $/kW
IRR = np.zeros( (len(CPP_range), len(CPE_range)) )
RTEs = [40,50,60,70,80,90]
Pout_max = 50 #MWe
Pin_max = 50 #MWe
SH = 4 #MWhe
Year = 2010
life = 20
exc_rate = pd.DataFrame({'Year':np.arange(2010,2020,1),
                         'AUD/USD':[0.92, 1.03, 1.04, 0.97, 0.9, 0.75, 0.74, 0.77, 0.75, 0.7] }).set_index('Year')



excel_file = ['c:/NextCloud/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx',
              '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx'][sys.platform == 'linux']
systems = pd.read_excel(excel_file, 'Summary',header=0, usecols='A:M')

fig, ax = plt.subplots(figsize=(16,18),nrows=3, ncols=2, gridspec_kw = {'wspace':0.05, 'hspace':0.1})
ax=np.ravel(ax)
font_size = 16


Region = 'SA'
db='storage_value_PF_simple'+'_%s.db'%(Region)
table = sm.list_tables(db)[0]
cols = [col[0] for col in sm.list_columns(db,table)]
data = sm.get_data(cols,table,db)

SV = data[(data['sh']==SH)&
          (data['loss']==0)&
          (data['cap']==14500)&
          (data['Pin_max']==Pin_max)&
          (data['Pout_max']==Pout_max)&
          (data['date_time']>Year)].set_index('date_time').sort_index()#['SV']



for r,RTE in enumerate(RTEs):
    
    SV = data[(data['sh']==SH)&
               (data['rte']==RTE)&
               (data['loss']==0)&
               (data['cap']==14500)&
               (data['Pin_max']==Pin_max)&
               (data['Pout_max']==Pout_max)&
               (data['date_time']>Year)].set_index('date_time').sort_index()['SV']

    SV_temp = pd.DataFrame({'SV': SV, 'xrate':exc_rate.loc[Year:2019]['AUD/USD']})
    SV_mean = (SV_temp['SV']*SV_temp['xrate']).mean()
    
    for cpe,CPE in enumerate(CPE_range):
        for cpp,CPP in enumerate(CPP_range):
            CAPEX = Pout_max*CPP*1e3+CPE*Pout_max*SH*1e3/(RTE/100)
            cashflows = np.insert(SV_mean*np.ones(life),0,-CAPEX)
            IRR[cpp,cpe] = round(np.irr(cashflows),4)

    X, Y = np.meshgrid(CPE_range, CPP_range)
    IRR[IRR>1]=1
    IRR[IRR<0]=-0.0001
    im = ax[r].contourf(X,Y, IRR, 100, cmap='Wistia')#
    ax[r].set_yscale('log')
    ax[r].set_xscale('log')
    L = [0,0.03,0.06,0.08,0.12,0.18,0.3]
    im2 = ax[r].contour(X,Y, IRR, colors='k',levels=L)
    
    clabels = ax[r].clabel(im2, inline_spacing=5, colors = 'k', fmt = '$%2.2f$', fontsize=font_size-2)
    fig.colorbar(im, ax=ax[r], format = '$%.2f$')

    ax[r].grid(which='both', axis='both')
    if r in [0,2,4]:
        ax[r].set_ylabel(r'$ \textrm{CPP}\; [\$/\textrm{kW}] $', fontsize=font_size)
    if r in [4,5]:
        ax[r].set_xlabel(r'$ \textrm{CPE}\; [\$/\textrm{kWh}] $', fontsize=font_size)
    if r in [1,3,5]:
        ax[r].tick_params(labelleft=False)
    if r in [0,1,2,3]:
        ax[r].tick_params(labelbottom=False)
    
    Systems = systems[(systems['Eff_low']*100<=RTE) & (systems['Eff_high']*100>=RTE)]
    for i in Systems.index:
        ax[r].errorbar(Systems.loc[i,'CPE_mean'],Systems.loc[i,'CPP_mean'],
                       xerr=np.array([[Systems.loc[i,'CPE_err-'],Systems.loc[i,'CPE_err+']]]).T, 
                       yerr=np.array([[Systems.loc[i,'CPP_err-'],Systems.loc[i,'CPP_err+']]]).T, 
                       marker=Systems.loc[i,'Marker'],
                       color=Systems.loc[i,'color'],
                       mec='b',ms=10, label = Systems.loc[i,'Type_ref']
                      ,ecolor='b', capsize=5,linestyle = 'None')
       
    ax[r].text(0.87,0.95,r'$\textrm{RTE=%d\%%}$'%(RTE),
              fontsize=font_size, transform=ax[r].transAxes,
               ha='center', bbox=dict(facecolor='w', edgecolor='k'), zorder=10)

    ax[r].set_ylim([CPP_range.min(),CPP_range.max()])
    ax[r].set_xlim([CPE_range.min(),CPE_range.max()])
    lg = ax[r].legend(loc='lower left', ncol=1,
                        fontsize=font_size-2,
                        edgecolor='k',
                        handletextpad=0.5)
        
plt.rc('ytick', labelsize=font_size)
plt.rc('xtick', labelsize=font_size)

# plt.savefig(savedir + 'IRR_CPP_CPE_%d-2019.png'%(Year+1), format='png', dpi=600, bbox_inches = "tight")
plt.show()

# IRR vs CPE and CPP with cap contracts for SA

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import matplotlib as mpl
import package.sql_manager as sm
import pdb
import matplotlib.gridspec as gridspec
import package.get_NEM_data as gd
%matplotlib inline
plt.rcParams['text.usetex'] = True
sep = ['\\','/'][sys.platform=='linux']
def er(x,y):
    return(y-av([x,y]))

CPE_range = np.concatenate((np.arange(1,10,1), np.arange(10,1100,5)),axis=0  ) # $/kWh
CPP_range = np.concatenate( (np.arange(10,100,2), np.arange(100,6000,10)),axis=0 ) # $/kW
IRR = np.zeros( (len(CPP_range), len(CPE_range)) )
RTEs = [40,50,60,70,80,90]
Pout_max = 50 #MWe
Pin_max = 50 #MWe
SH = 4 #MWhe
Year = 2015
life = 20

excel_file = ['c:/NextCloud/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx',
              '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx'][sys.platform == 'linux']
systems = pd.read_excel(excel_file, 'Summary',header=0, usecols='A:M')

fig, Axes = plt.subplots(figsize=(16,18),nrows=3, ncols=2, gridspec_kw = {'wspace':0.05, 'hspace':0.1})
Ax=np.ravel(Axes)
font_size = 16


Region = 'SA'
db='SV_NEM_cap_contract.db'
data = sm.get_data('N/A','N/A',db)

SV = data[(data.state==Region)&
          (data.sh==SH)&
          (data.loss==0)&
          (data.cap==14500)&
          (data.Pin_max==Pin_max)&
          (data.Pout_max==Pout_max)&
          (data.date_time>Year)].sort_values('date_time')#['SV']


# Convert the NEM SV to usd
SV['SV_total']= data[['SV_total','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
SV['SV']= data[['SV','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
# convert to 2015 USD
SV['SV'] = SV.apply(lambda x: gd.time_value(x['SV'],x['date_time'],2015,0.02 ),axis=1)
SV['SV_total'] = SV.apply(lambda x: gd.time_value(x['SV_total'],x['date_time'],2015,0.02 ),axis=1)


for r,RTE in enumerate(RTEs):
    i=int(np.divide(r,2))+1
    j=int(np.mod(r,2))+1
    SV_rte = SV[SV['rte']==RTE]['SV_total']
    SV_mean = SV_rte.mean()
    
    for cpe,CPE in enumerate(CPE_range):
        for cpp,CPP in enumerate(CPP_range):
            CAPEX = Pout_max*CPP*1e3+CPE*Pout_max*SH*1e3/(RTE/100)
            cashflows = np.insert(SV_mean*np.ones(life),0,-CAPEX)
            IRR[cpp,cpe] = round(np.irr(cashflows),4)

    X, Y = np.meshgrid(CPE_range, CPP_range)
    IRR[IRR>1.5]=1.5
    IRR[IRR<0]=-0.0001
    
    im = Ax[r].contourf(X,Y, IRR, 100, cmap='Wistia')#
    Ax[r].set_yscale('log')
    Ax[r].set_xscale('log')
    L = [0,0.03,0.06,0.08,0.12,0.18,0.3, 0.4, 0.5, 0.6]
    im2 = Ax[r].contour(X,Y, IRR, colors='k',levels=L)
    clabels = Ax[r].clabel(im2, inline_spacing=5, colors = 'k', fmt = '$%2.2f$', fontsize=font_size-2)

    Ax[r].grid(which='both', axis='both')
    if r in [0,2,4]:
        Ax[r].set_ylabel(r'CPP [$\$$/kW]', fontsize=font_size)
    if r in [4,5]:
        Ax[r].set_xlabel(r'CPE [$\$$/kWh]', fontsize=font_size)
    if r in [1,3,5]:
        Ax[r].tick_params(labelleft=False)
    if r in [0,1,2,3]:
        Ax[r].tick_params(labelbottom=False)
    
    Systems = systems[(systems['Eff_low']*100<=RTE) & (systems['Eff_high']*100>=RTE)]
    for i in Systems.index:
        Ax[r].errorbar(Systems.loc[i,'CPE_mean'],Systems.loc[i,'CPP_mean'],
                       xerr=np.array([[Systems.loc[i,'CPE_err-'],Systems.loc[i,'CPE_err+']]]).T, 
                       yerr=np.array([[Systems.loc[i,'CPP_err-'],Systems.loc[i,'CPP_err+']]]).T, 
                       marker=Systems.loc[i,'Marker'],
                       color=Systems.loc[i,'color'],
                       mec='b',ms=10, label = Systems.loc[i,'Type_ref']
                      ,ecolor='b', capsize=5,linestyle = 'None')
       
    Ax[r].text(0.87,0.93,'RTE=%d\%%'%(RTE),
              fontsize=font_size, transform=Ax[r].transAxes,
               ha='center', bbox=dict(facecolor='w', edgecolor='k'), zorder=10)

    Ax[r].set_ylim([CPP_range.min(),CPP_range.max()])
    Ax[r].set_xlim([CPE_range.min(),CPE_range.max()])
    lg = Ax[r].legend(loc='lower left', ncol=1,
                        fontsize=font_size-4,
                        edgecolor='k',
                        handletextpad=0.5,
                        framealpha=0.7)
cbar = fig.colorbar(im, ax=[Axes[0:,:2]], location='right', shrink=0.3, pad=0.01)

import matplotlib.ticker as tick
cbar.ax.yaxis.set_major_formatter(tick.FormatStrFormatter('%.1f'))
plt.rc('ytick', labelsize=font_size)
plt.rc('xtick', labelsize=font_size)

savedir = paperdir + 'Pictures%sIRR_CPE_CPP%s'%(sep,sep)

# plt.savefig(savedir + 'IRR_CPP_CPE_%d-2019 (SH=%d)_with_cap.png'%(Year+1,SH),dpi=600,  format='png',bbox_inches = "tight")
# plt.savefig(savedir + 'IRR_CPP_CPE_%d-2019 (SH=%d)_with_cap.svg'%(Year+1,SH),bbox_inches = "tight")
plt.show()

# IRR vs CPE and CPP with cap contracts for BGE

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import matplotlib as mpl
import package.sql_manager as sm
import pdb
import matplotlib.gridspec as gridspec
import package.get_NEM_data as gd
%matplotlib inline
plt.rcParams['text.usetex'] = True
sep = ['\\','/'][sys.platform=='linux']
def er(x,y):
    return(y-av([x,y]))

CPE_range = np.concatenate((np.arange(1,300,1), np.arange(300,1100,5)),axis=0  ) # $/kWh
CPP_range = np.concatenate( (np.arange(10,300,2), np.arange(300,6000,10)),axis=0 ) # $/kW

IRR = np.zeros( (len(CPP_range), len(CPE_range)) )
RTEs = [40,50,60,70,80,90]
Pout_max = 50 #MWe
Pin_max = 50 #MWe
SH = 4 #MWhe
Year = 2015
life = 20

excel_file = ['c:/NextCloud/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx',
              '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx'][sys.platform == 'linux']
systems = pd.read_excel(excel_file, 'Summary',header=0, usecols='A:M')

fig, Axes = plt.subplots(figsize=(16,18),nrows=3, ncols=2, gridspec_kw = {'wspace':0.05, 'hspace':0.1})
Ax=np.ravel(Axes)
font_size = 16


Region = 'BGE'
db='storage_value_arbi_capacity_PJM.db'
table = sm.list_tables(db)[0]
cols = [col[0] for col in sm.list_columns(db,table)]
data = sm.get_data(cols,table,db)

SV = data[(data['zone']==Region)&
          (data['sh']==SH)&
          (data['loss']==0)&
          (data['pin_max']==Pin_max)&
          (data['pout_max']==Pout_max)&
          (data['year']>Year)].sort_values('year')#['SV']


# convert to 2015 USD
SV['SV_arbi'] = SV.apply(lambda x: gd.time_value(x['SV_arbi'],x['year'],2015,0.02 ),axis=1)
SV['SV_total'] = SV.apply(lambda x: gd.time_value(x['SV_total'],x['year'],2015,0.02 ),axis=1)


for r,RTE in enumerate(RTEs):
    i=int(np.divide(r,2))+1
    j=int(np.mod(r,2))+1
    SV_rte = SV[SV['rte']==RTE]['SV_total']
    SV_mean = SV_rte.mean()
    
    for cpe,CPE in enumerate(CPE_range):
        for cpp,CPP in enumerate(CPP_range):
            CAPEX = Pout_max*CPP*1e3+CPE*Pout_max*SH*1e3/(RTE/100)
            cashflows = np.insert(SV_mean*np.ones(life),0,-CAPEX)
            IRR[cpp,cpe] = round(np.irr(cashflows),4)

    X, Y = np.meshgrid(CPE_range, CPP_range)
    IRR[IRR>1.5]=1.5
    IRR[IRR<0]=-0.0001
    
    im = Ax[r].contourf(X,Y, IRR, 100, cmap='Wistia')#
    Ax[r].set_yscale('log')
    Ax[r].set_xscale('log')
    L = [0,0.03,0.06,0.08,0.12,0.18,0.3, 0.4, 0.5, 0.6]
    im2 = Ax[r].contour(X,Y, IRR, colors='k',levels=L)
    clabels = Ax[r].clabel(im2, inline_spacing=5, colors = 'k', fmt = '$%2.2f$', fontsize=font_size-2)

    Ax[r].grid(which='both', axis='both')
    if r in [0,2,4]:
        Ax[r].set_ylabel(r'CPP [$\$$/kW]', fontsize=font_size)
    if r in [4,5]:
        Ax[r].set_xlabel(r'CPE [$\$$/kWh]', fontsize=font_size)
    if r in [1,3,5]:
        Ax[r].tick_params(labelleft=False)
    if r in [0,1,2,3]:
        Ax[r].tick_params(labelbottom=False)
    
    Systems = systems[(systems['Eff_low']*100<=RTE) & (systems['Eff_high']*100>=RTE)]
    for i in Systems.index:
        Ax[r].errorbar(Systems.loc[i,'CPE_mean'],Systems.loc[i,'CPP_mean'],
                       xerr=np.array([[Systems.loc[i,'CPE_err-'],Systems.loc[i,'CPE_err+']]]).T, 
                       yerr=np.array([[Systems.loc[i,'CPP_err-'],Systems.loc[i,'CPP_err+']]]).T, 
                       marker=Systems.loc[i,'Marker'],
                       color=Systems.loc[i,'color'],
                       mec='b',ms=10, label = Systems.loc[i,'Type_ref']
                      ,ecolor='b', capsize=5,linestyle = 'None')
       
    Ax[r].text(0.87,0.93,'RTE=%d\%%'%(RTE),
              fontsize=font_size, transform=Ax[r].transAxes,
               ha='center', bbox=dict(facecolor='w', edgecolor='k'), zorder=10)

    Ax[r].set_ylim([CPP_range.min(),CPP_range.max()])
    Ax[r].set_xlim([CPE_range.min(),CPE_range.max()])
    lg = Ax[r].legend(loc=['lower left','lower right'][r in [0,1]],
                      ncol=1,
                      fontsize=font_size-4,
                      edgecolor='k',
                      handletextpad=0.5,
                      framealpha=0.7)
cbar = fig.colorbar(im, ax=[Axes[0:,:2]], location='right', shrink=0.3, pad=0.01)

import matplotlib.ticker as tick
cbar.ax.yaxis.set_major_formatter(tick.FormatStrFormatter('%.1f'))
plt.rc('ytick', labelsize=font_size)
plt.rc('xtick', labelsize=font_size)

savedir = paperdir + 'Pictures%sIRR_CPE_CPP%s'%(sep,sep)

# plt.savefig(savedir + 'IRR_CPP_CPE_%s_%d-2019 (SH=%d)_with_cap.png'%(Region, Year+1,SH),dpi=600,  format='png',bbox_inches = "tight")
plt.savefig(savedir + 'IRR_CPP_CPE_%s_%d-2019 (SH=%d)_with_cap.svg'%(Region, Year+1,SH),bbox_inches = "tight")
plt.show()

# IRR vs sh for each technology

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm
import package.get_NEM_data as gd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pdb

sep = ['\\','/'][sys.platform=='linux']
Pout_max = 50 #MWe
Pin_max = 50 #MWe
life = 20

excel_file = ['c:/NextCloud/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx',
              '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx'][sys.platform == 'linux']
systems = pd.read_excel(excel_file, 'Summary',header=0, usecols='A:N')

Year = 2015
Region = 'SA'
db='SV_NEM_cap_contract.db'
data = sm.get_data('N/A','N/A',db)


Fig = make_subplots(rows=3, cols=2, shared_xaxes=True, vertical_spacing=0.02)
name_list = []
subplot_id = {'Li': (1,1),
              'NaS': (1,1),
              'PHS': (1,2),
              'LA': (1,1),
              'VRFB': (2,2),
              'ZnBr': (2,2),
              'HFC-AG': (2,1),
              'HFC-UG': (2,1),
              'HGT-AG': (2,1),
              'HGT-UG': (2,1),
              'CAES': (1,2),
              'HFC': (2,1),
              'PTES': (3,1),
              'MS': (3,2),
              'MS-sH2O': (3,2),
              'MS-sCO2': (3,2),
              'Si-TPV': (3,2),
              'Si-GT': (3,2),
              'LAES': (3,1)}
text_loc = {'black':18,
            'deepskyblue': 8,
            'tomato': 11,
            'magenta': 14,
            'green': 4,
            'gray': 16,
            'brown': 12,
           'lightcoral':8}
for i in systems.index[0:31]:
    IRR = []
    RTE = int(100*systems.loc[i,'Eff_high'])
    CPP = systems.loc[i,'CPP_mean']
    CPE = systems.loc[i,'CPE_mean']
    
    sv = data[(data.state==Region)&
              (data.loss==0)&
              (data.cap==14500)&
              (data.Pin_max==Pin_max)&
              (data.Pout_max==Pout_max)&
              (data.date_time>Year)&
              (data.cap_sampling=='annually')&
              (data.rte==RTE)].sort_values('date_time').copy()



    # Convert the NEM SV to usd
    sv['SV_total']= sv[['SV_total','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
#     sv['SV']= sv[['SV','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
    # convert to 2015 USD
#     sv['SV'] = sv.apply(lambda x: gd.time_value(x['SV'],x['date_time'],2015,0.02 ),axis=1)
    sv['SV_total'] = sv.apply(lambda x: gd.time_value(x['SV_total'],x['date_time'],2015,0.02 ),axis=1)
    
    SV = sv.groupby('sh').mean().sort_values('sh').reset_index()
    
    for s,SH in enumerate(SV.sh):
        CAPEX = Pout_max*CPP*1e3+CPE*Pout_max*SH*1e3/(RTE/100)
        rev = SV[SV.sh==SH]['SV_total'].values[0]
        cashflows = np.insert(rev*np.ones(life),0,-CAPEX)
        IRR.append ( round(np.irr(cashflows),4)  )
    s = systems.loc[i,'Type_ref']
    start = s.find('{')
    end = s.find('}')
    Name = s[start+1:end]
    Fig.add_trace(go.Scatter(x = SV.sh, y=IRR,
                             name = Name, legendgroup = Name, 
                             showlegend = [False,False][Name in name_list],
                             line=dict(color=systems.loc[i,'Line_Color'], 
                                       dash=['dash','solid'][int(systems.loc[i,'Proven'])])
                             ),
                  row=subplot_id[Name][0],col = subplot_id[Name][1])
    t = text_loc[systems.loc[i,'Line_Color']]
    Fig.add_annotation(x=SV.sh[t],y=IRR[t], text=s,
                       font=dict(size=12, color=systems.loc[i,'Line_Color']),
                       row=subplot_id[Name][0],col=subplot_id[Name][1], bgcolor="white")
    name_list.append(Name)
    
for Row in [1,2,3]:
    for Col in [1,2]:

        Fig.update_xaxes({'title':['','SH [Hours]'][Row==3],
                          'tick0': 0.5,
                          'tickvals':np.insert(np.arange(2,20,2.0),0,0.5,axis=0), 
                          'ticks':'outside',
                          'tickangle':0,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True},row=Row, col=Col)
        Fig.update_yaxes({'title':['IRR',''][Col==2],
                          'ticks':'outside',
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True},row=Row, col=Col)
        
Fig.update_layout(plot_bgcolor='white',
                  width=850, height=700,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')
    

savedir = paperdir + 'Pictures%sIRR_CPE_CPP%s'%(sep,sep)
# Fig.write_image(savedir + 'IRR_SH_%s_%d-2019.svg'%(Region, Year+1))
Fig.show()

# Storage value vs self discharge

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm
import package.get_NEM_data as gd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

sep = ['\\','/'][sys.platform=='linux']
PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

regions = ['SA', 'BGE']
RTEs = np.arange(100,20,-10)
SHours = np.arange(18,0,-1)
loss = 0
cap = 14500
Colors = ['violet','gold','mediumslateblue','orangered','mediumseagreen',
         'magenta','orange','skyblue']

years = [2014,2019]
Fig = make_subplots(rows=2, cols=2,
                    subplot_titles=(['%s (%d)'%(x, y) for x,y in zip(regions*2,np.repeat(years,2))]),
                    vertical_spacing=0.1)
for R,Region in enumerate(regions):
    if Region in NEM:
        db = 'storage_value_PF_simple_%s.db'%(Region)
        reg_type = 'state'
    if Region in PJM:
        db = 'storage_value_PF_simple_PJM.db'
        reg_type = 'zone'
        
    table = 'storage_value'
    cols = [x[0] for x in sm.list_columns(db,table)]
    data = sm.get_data(cols,table, db)
    for i,year in enumerate(years):
        for r,RTE in enumerate(RTEs):
            SV = data[(data['date_time']==year) &
                      (data[reg_type]==Region)&
                      (data['rte']==RTE) &
                      (data['loss']==loss)&
                      (data['Pin_max']==50)&
                      (data['Pout_max']==50)&
                      (data['sh']<19)]
            if reg_type=='state':
                SV=SV[SV['cap']==cap]
                SV['SV']= SV[['SV','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)

            SV = SV.sort_values('sh', axis=0)
            
            Fig.add_trace(go.Scatter(x=SV['sh'],
                                 y=SV['SV']/(50),
                                 line = dict(width=1),
                                 name='%d%%'%(RTEs[r]),
                                 marker_color=Colors[r],
                                 marker_line_color='black',
                                 legendgroup='%d'%(RTE),
                                 showlegend = [False,True][(R==1)&(i==1)]),
                                 row=i+1, col=R+1)

for i in [1,2]:
    for j in [1,2]:
        Fig.update_xaxes({'title':['','SH [Hours]'][i==2],
                          'tickmode': 'linear',
                          'tick0': 0,
                          'dtick': 2,
                          'showticklabels':[False,True][i==2],
                          'ticks':'outside',
                          'tickangle':0,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        Fig.update_yaxes({'title':['SV [$/MW-year]',''][j==2],
                          'ticks':'outside',
                          'nticks':14,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        
Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='v',
                  legend=dict(x=1.01, y=0),
                  legend_title_text = '         RTE',
                  width=900, height=600,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')
savedir = paperdir + 'Pictures%sStorage value vs SH_line chart%s'%(sep,sep)
Fig.show()
# Fig.write_image(savedir + 'SV_SH_%s.svg'%("-".join(regions)))


In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm
import package.get_NEM_data as gd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

sep = ['\\','/'][sys.platform=='linux']
PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

regions = ['SA', 'BGE']
RTEs = np.arange(30,100,20)
SH = 10
loss = np.arange(0,11,1)
cap = 14500

Colors = ['violet','gold','mediumslateblue','orangered','mediumseagreen',
         'magenta','orange','skyblue']
# plt.close("all")
# font_size = 16
years = np.arange(2010,2019,4)
# fig, ax = plt.subplots(figsize=(16,12),nrows=len(years), ncols=2, gridspec_kw = {'wspace':0.15, 'hspace':0.1})
Fig = make_subplots(rows=len(years), cols=2,
                    subplot_titles=(['%s (%d)'%(x, y) for x,y in zip(regions*3,np.repeat(years,2))]),
                    vertical_spacing=0.1)


for r,Region in enumerate(regions):
    if Region in NEM:
        db = 'storage_value_PF_simple_%s.db'%(Region)
        reg_type = 'state'
    if Region in PJM:
        db = 'storage_value_PF_simple_PJM.db'
        reg_type = 'zone'
        
    table = 'storage_value'
    cols = [x[0] for x in sm.list_columns(db,table)]
    data = sm.get_data(cols,table, db)
    for i,Year in enumerate(years):
        for j,RTE in enumerate(RTEs):
            SV = data[(data['date_time']==Year)&
                      (data['rte']==RTE) &
                      (data['sh']==SH) &
                      (data['Pout_max']==50) &
                      (data['Pin_max']==50)]
            if Region in NEM:
                 SV = SV[SV['cap']==cap]
            SV = SV.sort_values('loss', axis=0)
            normaliser = SV['SV'].max()
            Fig.add_trace(go.Scatter(x=SV['loss'],
                                 y=SV['SV']/normaliser,
                                 line = dict(width=1),
                                 name='%d%%'%(RTE),
                                 marker_color=Colors[j],
                                 marker_line_color='black',
                                 legendgroup='%d'%(RTE),
                                 showlegend = [False,True][(r==1)&(i==2)]),
                                 row=i+1, col=r+1)
            
for i in np.arange(1,len(years)+1):
    for j in [1,2]:
        Fig.update_xaxes({'title':['','Self discharge [%/day]'][i==3],
                          'tickmode': 'linear',
                          'autorange':False,
                          'range':[0,11],
                          'dtick': 1,
                          'showticklabels':[False,True][i==3],
                          'ticks':'outside',
                          'tickangle':0,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        Fig.update_yaxes({'title':['SV (normalised)',''][j==2],
                          'ticks':'outside',
                          'autorange':False,
                          'range':[[0.7,0.85][j==1],1.02],
                          'dtick':[0.05,0.1][j==2],
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        
Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='h',
                  legend=dict(x=0.3, y=1.1),
                  legend_title_text = '',
                  width=900, height=800,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')

paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\\"
savedir = paperdir + 'Pictures%sStorage value vs loss%s'%(sep,sep)
Fig.write_image(savedir + 'SV_vs_loss_%s.png'%("-".join(regions)))
Fig.show()

# Plot the histogram of the length of storage charging and discharging (NEM)
This result shows that how many times the storage runs for a certain amount of time when it starts to charge or discharge.

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
sep = ['\\','/'][sys.platform=='linux']

import numpy as np
import multiprocessing as mp
import pandas as pd
from package.optimisation import calc_S_perfect_foresight
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import package.storage_analysis as sa

PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

regions=['SA','BGE']
years = [2010,2019]
rtes = [40,90]
Fig = make_subplots(rows=2,cols=2,
                    subplot_titles=['%s-%d%%'%(x,y) for x,y in zip(2*regions,np.repeat(rtes,2))],
                    shared_xaxes=True, 
                    shared_yaxes=True, 
                    vertical_spacing=0.1,
                    horizontal_spacing=0.02,
                   )
colors = ['tomato', 'deepskyblue']

for z,Region in enumerate(regions):
    data = sm.get_data('N/A','N/A','storage_value_perfect_%s.db'%(Region))
    data.date_time = pd.to_datetime(data.date_time)
    for r,RTE in enumerate(rtes):
        for y,Year in enumerate(years):
            if Region in NEM:
                dt=0.5
            if Region in PJM:
                dt=1
                data['Pout']=data.Pout_da+data.Pout_rt
                
                
            Data = data[(data.date_time.dt.year==Year)&
                        (data.rte==RTE)]
            profile = Data.set_index('date_time').Pout
            result = sa.operation_distribution(profile)

            Fig.add_trace(go.Histogram(x=dt*np.array(result),
                                      marker=dict(color=colors[y], line=dict(width=0.75, color='black')),
                                      name=str(Year),
                                      showlegend=[False,True][(r*z)==1],
                                      offsetgroup=Year)
                                       ,row=r+1, col=z+1)

for i in [1,2]:
    for j in [1,2]:
        Fig.update_xaxes({'title':['Duration [hr]',''][i==1],
                          'tickmode': 'linear',
                          'ticks':'outside',
                          'dtick':[1,2][j==2],
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        Fig.update_yaxes({'title':['Counts',''][j==2],
                          'ticks':'outside',
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)

Fig.update_layout(plot_bgcolor='white',
                  width=900, height=600,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='group',
                  bargap=0.2,
                  legend=dict(orientation="h",yanchor="bottom",
                              y=1.05,xanchor="center",x=0.5)
                  )

paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\\"
savedir = paperdir + 'Pictures%sStorage operation duration%s'%(sep,sep)

# Fig.write_image(savedir + 'discharging duration_%s.png'%("-".join(regions)))
Fig.show()

# pot the Pin and Put time series

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm
from datetime import timedelta, time
import package.get_NEM_data as gd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

region = 'BGE'
data = sm.get_data('N/A','N/A','storage_value_perfect_%s.db'%(region))
data.date_time = pd.to_datetime(data.date_time)
if Region in PJM:
    dt=1
    data['Pout']=data.Pout_da+data.Pout_rt
    data['Pin']=data.Pin_da+data.Pin_rt
            
Data = data[(data.rte==90)&
            (data.date_time.dt.year==2019)].sort_values('date_time')

import pandas as pd
import plotly.graph_objects as go
Fig = go.Figure()

Fig.add_trace(go.Scatter(x=Data.date_time, y=Data.Pin,line_shape='hv',
               yaxis='y2',name='Pin'))

Fig.add_trace(go.Scatter(x=Data.date_time, y=Data.Pout,
              line_shape='hv',yaxis='y2',name='Pout'))

Fig.update_layout(
    xaxis=dict(rangeselector=dict(buttons=list([
                dict(count=1,label="1d",step="day",stepmode="backward"),
                dict(count=7,label="1w",step="day",stepmode="backward"),
                dict(count=1,label="1m",step="month",stepmode="backward"),
                dict(count=1,label="1y",step="year",stepmode="backward"),
                dict(step="all")])
                                 ),
        rangeslider=dict(visible=True),type="date"),
   yaxis=dict(title='Q [MWh]'),
   yaxis2=dict(title='P [MW]',overlaying='y',side='right')
)

paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\\"
savedir = paperdir + 'Pictures%sStorage operation duration%s'%(sep,sep)

# Fig.write_image(savedir + 'charge_discharge_cycles%s.png'%("-".join(regions)))
Fig.show()

# Plot the storage operation duration for BGE

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
import package.sql_manager as sm
import pandas as pd

db='storage_value_perfect_BGE.db'
data=sm.get_data('N/A','N/A',db)
data.date_time=pd.to_datetime(data.date_time)

Data = data[data.date_time.dt.year==2010].copy()
Data['Pout']=Data.Pout_da+Data.Pout_rt
Data[['date_time','Pout']]


# Generate SV vs Loss table

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import package.sql_manager as sm
import pdb

sep = ['\\','/'][sys.platform=='linux']

savedir = paperdir + 'Pictures%sStorage value vs loss%s'%(sep,sep)

States = ['SA','NSW']
RTEs = np.arange(30,100,20)
SH = 10
loss = np.arange(0,11,1)
cap = 14500
years = np.arange(2010,2020,1)
for Year in years:
    sv_loss = pd.DataFrame(columns=[np.append(['ratio', 'state', 'rte'], [str(y) for y in years])])
    sv_loss['ratio']=np.repeat(['SV10:0-30%','SV10:0-50%','SV10:0-70%','SV10:0-90%'],2)


    for s,state in enumerate(States):
        db = 'storage_value_PF_simple_%s.db'%(state)
        table = 'storage_value'
        cols = [x[0] for x in sm.list_columns(db,table)]
        data = sm.get_data(cols,table, db)
        data = data[(data['Pin_max']==50)&((data['Pout_max']==50))]
        for i,RTE in enumerate(RTEs):    

            SV = data[(data['date_time']==int(Year)) &
                          (data['cap']==cap) &
                          (data['sh']==SH)]
            SV_10 = data[(data['cap']==cap) &
              (data['sh']==SH)&
              (data['rte']==RTE)&
              (data['loss']==10)].sort_values('date_time',axis=0)

            SV_0 = data[(data['cap']==cap) &
              (data['sh']==SH)&
              (data['rte']==RTE)&
              (data['loss']==0)].sort_values('date_time',axis=0)
#             pdb.set_trace()
            ratio = SV_10.set_index('date_time')['SV']/SV_0.set_index('date_time')['SV']
            for y,year in enumerate([str(s) for s in years]):
                sv_loss.loc[2*i+s][year]=ratio.tolist()[y]
                sv_loss.loc[2*i+s]['rte']=RTE
                sv_loss.loc[2*i+s]['state']=state
    #         sv_loss = sv_loss.append({
    #             'state': state,
    #             'rte': RTE
    #         }
    #             , ignore_index=True)

sv_loss

            

# Plot SV vs Pin and Pout

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm
import pdb
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sep = ['\\','/'][sys.platform=='linux']


Pin_maxs = np.concatenate((np.arange(1,5,1),np.arange(5,55,5)),axis=0)
Pout_maxs = Pin_maxs

def f(x,y):
    return np.array(DATA[(DATA['Pout_max']==y)&
                        (DATA['Pin_max']==x)]['SV']).mean()
g = np.vectorize(f)
X, Y = np.meshgrid(Pin_maxs, Pout_maxs)

font_size = 16
states = ['SA']
RTEs = [40,90]
SH=10

Fig = make_subplots(rows=1,cols=2,
                       subplot_titles=['%s (RTE=%s%%)'%(x,y) for x,y in zip(states*2,RTEs)])



for s,State in enumerate(states):
    db= 'storage_value_PF_simple_%s.db'%(State)
    table = sm.list_tables(db)[0]
    cols = [col[0] for col in sm.list_columns(db,table)]
    data = sm.get_data(cols,table,db)
    Year_sample = 2015
    for r,RTE in enumerate(RTEs):
        DATA = data[(data['rte']==RTE)&
                    (data['sh']==SH)&
                    (data['loss']==0)&
                    (data['date_time']>Year_sample)&
                    (data['cap']==14500)]#.sort_values('date_time')
        
        SV = g(X,Y)
        SV_norm = SV/SV.max()
        L = np.linspace(0, 1,11)
        Fig.add_trace(go.Contour(z=SV_norm,
                                 x=Pin_maxs,
                                 y =Pout_maxs,
                                 line_smoothing=0.85,
                                 contours=dict(
                                 coloring ='heatmap',
                                 showlabels = True, # show labels on contours
                                 labelfont = dict( # label font properties
                                                    size = 12,
                                                    color = 'white')),
                                 colorbar=dict(
                                                title='SV (normalised)', # title here
                                                titleside='right',
                                                titlefont=dict(size=14),
                                                )
                                
                                ),   row=s+1, col=r+1)


for i in [1]:
    for j in [1,2]:
        Fig.update_xaxes({'title':['',r'$\textrm{P}_{\textrm{ch}}^{\textrm{max}} \textrm{[MW]}$'][i==1],
                          'tickmode': 'linear',
                          'tick0': 0,
                          'dtick': 5,
                          'showticklabels':[False,True][i==1],
                          'ticks':'outside',
                          'tickangle':0,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'white',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        Fig.update_yaxes({'title':[r'$\textrm{P}_{\textrm{disch}}^{\textrm{max}} \textrm{[MW]}$',''][j==2],
                          'ticks':'outside',
                          'dtick':5,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'white',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        
Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='v',
                  legend=dict(x=1.01, y=0),
                  legend_title_text = '         RTE',
                  width=900, height=400,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')

paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\\"
savedir = paperdir + 'Pictures%sStorage value vs Pin_Pout%s'%(sep,sep)
# Fig.write_image(savedir + 'SV_Pch_Pdisch.png')

Fig.show()

# Line plot SV as a function of Pch

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm
import pdb
import plotly.graph_objects as go
from plotly.subplots import make_subplots
sep = ['\\','/'][sys.platform=='linux']



Pout_maxs = [10,50]

State = 'SA'
RTEs = [40,90]
SH=10

db= 'storage_value_PF_simple_%s.db'%(State)
data = sm.get_data('N/A','N/A',db)
Fig = make_subplots(rows=1,cols=2,
                    subplot_titles=['%s (RTE=%s%%)'%(x,y) for x,y in zip([State]*2,RTEs)])
Colors = ['orangered','violet','gold','mediumslateblue',
         'magenta','orange','deepskyblue','mediumseagreen']

for r,RTE in enumerate(RTEs):
    Data = data[(data.sh==10)&
                (data.rte==RTE)&
                (data.cap==14500)&
                (data.loss==0)&
                (data.Pout_max.isin(Pout_maxs))&
                (data.date_time>2015)].copy()
    Data['SV_norm']=Data.SV/Data.SV.max()
    for p,Pout in enumerate(Pout_maxs):
        DATA = Data[Data.Pout_max==Pout]
        Fig.add_trace(go.Scatter(
                                x= DATA.groupby(['Pin_max'])['Pin_max'].mean(),
                                y= DATA.groupby(['Pin_max'])['SV_norm'].mean(),
                                error_y=dict(
                                            type='data',
                                            thickness=1.25,
                                            symmetric=False,
                                            array=(DATA.groupby(['Pin_max'])['SV_norm'].max()-
                                                   DATA.groupby(['Pin_max'])['SV_norm'].mean()),
                                            arrayminus= DATA.groupby(['Pin_max'])['SV_norm'].mean()-
                                                        DATA.groupby(['Pin_max'])['SV_norm'].min(),
#                                             array=(DATA.groupby(['Pin_max'])['SV_norm'].std()).tolist(),
                                            visible=True),
                                line = dict(color=Colors[p],width=1),
                                name=str(Pout),
                                marker=dict(color=Colors[p],line_width=1, size=7),
                                legendgroup=str(Pout),
                                showlegend = [False,True][(r==1)]),
                                row=1, col=r+1)
                 
                             


for i in [1,2]:
    for j in [1,2]:
        Fig.update_xaxes({'title':['',r'$\text{P}_{\text{ch}}^{\text{max}}\text{ [MW]}$ '][i==1],
                          'tickmode': 'linear',
                          'tick0': 0,
                          'dtick': 5,
                          'showticklabels':[False,True][i==1],
                          'ticks':'outside',
                          'tickangle':0,
                          'showgrid':False,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        Fig.update_yaxes({'title':[r'$\overline{\text{SV}}$',''][j==2],
                          'ticks':'outside',
                          'dtick':0.1,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        
Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='h',
                  legend=dict(x=0.5, y=1.02,xanchor='center',yanchor='bottom',
                             title_text=r'$\text{P}_{\text{disch}}^{\text{max}}\text{[MW]}:\; $',
                             title_font_size=13),
                  
                  width=900, height=400,
                  margin=dict(l=10, r=10, t=20, b=10),
                  )

paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\\"
savedir = paperdir + 'Pictures%sStorage value vs Pin_Pout%s'%(sep,sep)
# Fig.write_image(savedir + 'SV_Pch_Pdisch=10_50.png')
Fig.show()